# Mapboxgl Python Library for location data visualizaiton

https://github.com/mapbox/mapboxgl-jupyter

### Requirements

These examples require the installation of the following python modules

```
pip install mapboxgl
pip install pandas
pip install pysal
```

In [9]:
import pysal.esda.mapclassify as mapclassify
import pandas as pd
import os
from mapboxgl.utils import *
from mapboxgl.viz import *

In [10]:
df = pd.read_csv('points.csv').round(3)
df.head(5)

,Avg Total Payments,Avg Covered Charges,Total Discharges,Avg Medicare Payments,admin1_id,Provider Id,admin2_id,lon,lat,date
0,8749.025,35247.028,58.750,7678.214,USA101,10001,USA201069,-85.363,31.216,1/1/14 12:00 AM
1,6812.131,16451.092,28.959,5793.631,USA101,10005,USA201119,-88.143,32.453,1/2/14 12:00 AM
2,8197.238,36942.357,45.360,7145.960,USA101,10006,USA201077,-87.683,34.794,1/3/14 12:00 AM
3,4860.829,12079.537,27.409,4047.025,USA101,10007,USA201039,-86.255,31.292,1/4/14 12:00 AM
4,5898.137,16148.752,17.889,4963.548,USA101,10008,USA201041,-86.265,31.694,1/5/14 12:00 AM


## Set your Mapbox access token.  
### Set a  `MAPBOX_ACCESS_TOKEN` environment variable or copy/paste your token from https://www.mapbox.com/studio/account/tokens/


In [11]:
# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

## Create a visualization from a Pandas dataframe

In [12]:
# Create a geojson file export from the current dataframe
df_to_geojson(df, filename='points1.geojson',
              properties=['Avg Medicare Payments', 'Avg Covered Charges', 'date'], 
                     lat='lat', lon='lon', precision=3)

{'feature_count': 3173, 'filename': 'points1.geojson', 'type': 'file'}

In [13]:
# Generate data breaks and color stops from colorBrewer
color_breaks = mapclassify.Natural_Breaks(df['Avg Medicare Payments'], k=8, initial=0).bins
color_stops = create_color_stops(color_breaks, colors='YlGnBu')

# Create the viz from the dataframe
viz = CircleViz('points1.geojson',
                access_token=token, 
                height='400px',
                color_property = "Avg Medicare Payments",
                color_stops = color_stops,
                center = (-95, 40),
                zoom = 3,
                below_layer = 'waterway-label')

viz.show()

## Add labels to the viz

In [14]:
viz.label_property = "Avg Medicare Payments"
viz.show()

## Change viz data property and color scale

In [15]:
# Generate a new data domain breaks and a new color palette from colorBrewer2
color_breaks = mapclassify.Natural_Breaks(df['Avg Covered Charges'], k=8, initial=0).bins
color_stops = create_color_stops(color_breaks, colors='YlOrRd')

# Show the viz
viz.color_property='Avg Covered Charges'
viz.color_stops=color_stops
viz.show()

### Change the viz map style

In [16]:
viz.style_url='mapbox://styles/mapbox/dark-v9?optimize=true'
viz.show()

## Create a graduated cricle viz based on two data properties

In [17]:
# Generate data breaks and color stops from colorBrewer
color_breaks = mapclassify.Natural_Breaks(df['Avg Covered Charges'], k=8, initial=0).bins
color_stops = create_color_stops(color_breaks, colors='Spectral')

# Generate radius breaks from data domain and circle-radius range
radius_breaks = mapclassify.Natural_Breaks(df["Avg Medicare Payments"], k=8, initial=0).bins
radius_stops = create_radius_stops(radius_breaks, 1, 10)

# Create the viz
viz2 = GraduatedCircleViz('points1.geojson', 
                          access_token=token,
                          color_property = "Avg Covered Charges",
                          color_stops = color_stops,
                          radius_property = "Avg Medicare Payments",
                          radius_stops = radius_stops,
                          center = (-95, 40),
                          zoom = 3,
                          below_layer = 'waterway-label')

viz2.show()

## Create a heatmap viz

In [18]:
#Create a heatmap 
heatmap_color_stops = create_color_stops([0.01,0.25,0.5,0.75,1], colors='RdPu')
heatmap_radius_stops = [[0,1], [15, 40]] #increase radius with zoom

color_breaks = mapclassify.Natural_Breaks(df['Avg Medicare Payments'], k=8, initial=0).bins
color_stops = create_color_stops(color_breaks, colors='Spectral')

heatmap_weight_stops = create_weight_stops(color_breaks)

#Create a heatmap 
viz3 = HeatmapViz('points1.geojson', 
                  access_token=token,
                  weight_property = "Avg Medicare Payments",
                  weight_stops = heatmap_weight_stops,
                  color_stops = heatmap_color_stops,
                  radius_stops = heatmap_radius_stops,
                  opacity = 0.9,
                  center = (-95, 40),
                  zoom = 3,
                  below_layer='waterway-label'
                 )

viz3.show()

## Create a clustered circle map

In [19]:
#Create a clustered circle map
color_stops = create_color_stops([1,10,50,100], colors='BrBG')

viz4 = ClusteredCircleViz('points1.geojson', 
                  access_token=token,
                  color_stops = color_stops,
                  radius_stops = [[1,5], [10, 10], [50, 15], [100, 20]],
                  cluster_maxzoom = 10,
                  cluster_radius = 30,
                  opacity = 0.9,
                  center = (-95, 40),
                  zoom = 3
                 )

viz4.show()

# Save our viz to an HTML file for distribution
### Note
Viz export contains a reference to the data in this visualization.  Serve data from the same directory as the HTML file to vis your visualization.

In [20]:
with open('viz4.html', 'w') as f:
    f.write(viz4.create_html())

### Run exported HTML example

Python2: `python -m SimpleHTTPServer 8080`

Python3: `python3 -m http.server 8080`

Now navigate your browser to `http://localhost:8080/viz4.html` to see the viz